# Analysing Hillslope Morphology at the Landscape Scale

Hillslope morphology reflects the competition between surface uplift and river erosion, which sets the baselevel conditions for hillslopes. In lab 2 you ran a simple numerical model in order to explore how changes in uplift and erosion influence hillslope development. In this lab, you will look at some hillslope morphology data from a real landscape in order to try and interpret the distribution of erosion in order to infer rates of tectonic processes. The data has already been generated for you by running software developed by Hurst et al. (2012) and Grieve et al (2016) to extract information about hillslope morphology from digitial elevation models (DEMs). This software is slow and was run on a very large dataset so the results have been provided in the data download from Moodle. The aims of this lab are:
* Follow and execute python code to analyse hillslope morphology extracted from a DEM.
* Further understand the concept of steady-state and transience in hillslope morphology
* Quantify metrics of dimensionless hillslope morphology
* Analyse hillslope morphology at a landscape scale to evaluate the distribution of erosion/uplift.
* Create and explain figures that show variation in hillslope form in a tectonically active landscape

## Bolinas
The landscape you will work on is the Bolinas Ridgeline, which runs adjacent to the San Andreas Fault in California, just north of San Francisco, over the Golden Gate Bridge. You will analyse hillslope morphology in 40 catchments along the fault that drain towards the fault. These catchments are of similar size, but relatively small, no more than  ~4 km in length. The elevation of the landscape generally increases from NW to SE.

![title](images/basemap.png "ShowMyImage")

The bedrock beneath these catchments is uniform, consisting of poorly-consolidated metasedimentary rocks (sandstones) of the Franciscan Fm. The key point here is that there should be little variation in channel or hillslope form due to differences in bedrock type. The hillslopes here are all soil-mantled, and thus you can assume transport-limited conditions and that the nonlinear sediment flux law is appropriate to govern downslope soil flux on the hillslopes.

Previous work by Kirby et al (2007) has examined the morphology of the river profiles and found that rivers get progressively steeper, with increasing $M_{\chi}$ from NW to SE. The parameter $M_{\chi}$ is an estimate of channel slope normalised to drainage area, and is considered a good proxy for erosion rate.

![title](images/chi_steepness_map.png "ShowMyImage")

Kirby et al (2007) suggested that channel erosion rates increased from NW to SE and thus inferred that there was an increase in tectonic uplift rate along this landform. This is important because the high uplift rates make this site a potential nucleus for future earthquakes and San Francisco has a prominent history of being hit by large earthquakes. Millions of people live in the Bay Area, which also has one of the USA's biggest ports, and there is a great deal of industry near by from hi-tech manufacturing to financial services. We will explore whether hillslope morphology reflects the patterns of erosion inferred from the channel network.

## Python
The programming language we are using in this lab is called Python.  No prior knowledge of programming is required for this lab. Learning how to be a programmer is not the aim! However, this sort of scientific computing is becoming more common place in research and consultancy, so it won't do you any harm to see it in action. Python is multifunctional, for example it can interface with ArcGIS (the software used in the previous lab) to automate workflows. In the last lab we saw how it can be used to construct numerical models and visualise results. In this lab we'll use it to visualise the results of analyses performed on topographic data.

**To run a code block, click in a cell, hold down shift, and press enter.** An asterisk in square brackets `In [*]:` will appear while the code is being executed, and this will change to a number `In [1]:` when the code is finished. *The order in which you execute the code blocks matters, they must be run in sequence.* 

Inside blocks of python code there are comments indicated by lines that start with `#`. These lines are not computer code but rather comments providing information about what the code is doing to help you follow along. Before we get started we need to tell python which tools we want to use (these are called modules):


In [ ]:
# import modules for numerical calculations, for plotting, and for data manipulation
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd

# import operating system functions
import os

# tell python to allow plotting to occur within the page
%matplotlib inline

# Customise figure style, changing the font to arial and the size to 16pt
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Arial']})
rc('font',size=12)

Before we get started we need to define the workspace we are working in so that python knows where to go to look for files and data.

In [ ]:
# Get the current directory
DataDirectory = os.getcwd()+"\\Data\\"

# Set the project name
ProjectName = "Bolinas"

## Load the hillslope data
The file named **Bolinas_HilltopData.csv** contains the results of all the hillslope traces collected along the Bolinas ridge. Each trace starts at a hilltop and follows the steepest path down to an adjacent channel. Open up this file in excel and take a look at the data table. Information about the location of the hilltop ($X$,$Y$,$i$,$j$) are included, as are the location where the traces stopped in the channel network ($a$,$b$,*StreamID*). For our purposes, the hilltop curvature $C_{HT}$, hillslope length $L_H$, hillslope relief $R$ and average hillslope gradient $S$ are all recorded, and this is the data we wil be using mostly. The code below reads this data into Python so that we can explore it, and then checks the data for errors.

In [ ]:
# load the hillslopes data
Filename = ProjectName+'_HilltopData.csv'
HillslopeData = pd.read_csv(DataDirectory+Filename)

# drop any rows with no data (hillslope traces to outside the study domain)
# or with value of -9999 for Basin ID
HillslopeData = HillslopeData.dropna()
HillslopeData = HillslopeData[HillslopeData.BasinID != -9999]

## Determine *S<sub>C</sub>*
The critical slope gradient *S<sub>C</sub>* is an important parameter to constrain. This can be done based on topographic data following the work of Grieve et al. (2016). Recall from equation X that *S<sub>C</sub>* is the gradient toward which hillslope sediment transport accelerates rapidly toward infinity. Therefore it should not be possible for a hillslope to be steeper than *S<sub>C</sub>*. 

We can determine *S<sub>C</sub>* by looking at the percentage of hillslopes that are steeper than a range of *S<sub>C</sub>* values. We will arbritarily assume that *S<sub>C</sub>* is the value at which >99.5% of hillslopes in the entire Bolinas landscape have a lower slope.

In [ ]:
# set up range of S_c values to test from 0.5 to 1.
Sc_test = np.arange(0.5,1.,0.01)

# set up empty array for results    
Percent_Less_Than = np.zeros(len(Sc_test))
    
NoHillslopes = len(HillslopeData)    

#loop across the 
for i, Sc in enumerate(Sc_test):
            
    # get max hillslope relief
    R_test = HillslopeData.Lh*Sc
        
    #compare to actual hillslope relief and count how many fall below max line
    #first for segments
    BelowMask = HillslopeData.R < R_test
    NumberBelow = np.sum(BelowMask)
    Percent_Less_Than[i] = 100.*float(NumberBelow)/float(NoHillslopes)

# find the location of the 99.5th percentile    
ind = np.argmin(np.abs(99-Percent_Less_Than))

# SET S_C BASED ON THE RESULTS
Sc = Sc_test[ind]

# plot the resulting curve
plt.plot(Sc_test,Percent_Less_Than,'k-',color=[0.5,0.5,0.5])

# plot the percentile
plt.plot([0.5,Sc_test[ind],Sc_test[ind]],[99,99,0.],'r--',lw=0.5)

#plot the value of Sc
plt.text(Sc_test[ind],85,"$S_C = $"+str(Sc_test[ind]),rotation=-90)

#label the axes
plt.xlabel("$S_C$")
plt.ylabel("Percent with lower Relief")
plt.ylim(70,100)
plt.xlim(0.5,1.0)
plt.tight_layout()
plt.savefig(ProjectName+"Sc_Percent.pdf")

## Hillslope morphology within an individual catchment
During the first part of this lab, you will explore hillslope morphology within an individual catchment along the Bolinas ridge. There are 40 catchments to choose from so you can explore the catchments to decide which you think might be the most interesting. The catchments are numbered from NW to SE.

![title](images/basemap_numbered.png "ShowMyImage")

### Hillslope Traces

The data you will be working with consists of individual hillslope traces from every hilltop down the path of steepest descent to a neigbouring channel pixel. Only the hilltops falling within or on the boundary of the 41 catchments pictured above have been extracted. There are over 150,000 individual hillslope traces, so this is alot of data to analyse and will require us to group the data spatially and calculate summary statistics. The image below shows you some examples of individual hillslope traces across a DEM. The hilltops are marked in black and the channels in blue, the hillslope traces are depicted with red lines.

![title](images/grieve_traces.png "ShowMyImage")

### Plot Histograms of Hillslope Morphology

You will start by looking at the distribution of hillslope morphology within your catchments by plotting histograms of the important hillslope metrics: hillslope length, slope, relief and hilltop curvature:

In [ ]:
# SET THE BASIN NUMBER YOU WISH TO WORK WITH HERE
Basin = 24

# get the hillslope data only for the basin you are working on
Basins = np.sort(HillslopeData.BasinID.unique())
BasinHillslopeData = HillslopeData[HillslopeData.BasinID == Basins[Basin]]
    
# setup the figure
# you can modify the figure size here if you wish
fig_width_inches = 6.6
fig_height_inches = 6.6
plt.figure(figsize=(fig_width_inches,fig_height_inches))

# this figure is going to have four parts, called subplots
# we create a subplot by providing a 3 digit number, 
# the first two digits refer to the dimensions of the plot (2x2 subplots)
# the third number refers to which of these we want to populate

# HILLSLOPE LENGTH
plt.subplot(221)
# generate the frequency data for a histogram for hillslope length
freq, bin_edges = np.histogram(BasinHillslopeData.Lh,bins=np.arange(0,400,20.))
# normalise the frequency data to the largest value in the dataset
freq_norm = freq.astype(np.float)/float(np.max(freq))
# plot the results as a bar chart
plt.bar(bin_edges[:-1],freq_norm,width=20,align='edge',edgecolor='k',linewidth=0.5,color='steelblue')
plt.xlabel("Hillslope Length (m)")
plt.ylabel("Normalised Frequency")
plt.xlim(0,400)
    
# HILLSLOPE GRADIENT
plt.subplot(222)
freq, bin_edges = np.histogram(BasinHillslopeData.S,bins=np.arange(0,0.8,0.04))
freq_norm = freq.astype(np.float)/float(np.max(freq))
plt.bar(bin_edges[:-1],freq_norm,width=0.04,align='edge',edgecolor='k',linewidth=0.5,color='thistle')
plt.xlabel("Mean Slope (m/m)")
plt.ylabel("Normalised Frequency")
plt.xlim(0,0.8)
    
# HILLTOP CURVATURE
plt.subplot(223)
freq, bin_edges = np.histogram(BasinHillslopeData.R,bins=np.arange(0,200.,10.))
freq_norm = freq.astype(np.float)/float(np.max(freq))
plt.bar(bin_edges[:-1],freq_norm,width=10.,align='edge',edgecolor='k',linewidth=0.5,color='sandybrown')
plt.xlabel("Hillslope Relief (m)")
plt.ylabel("Normalised Frequency")
plt.xlim(0,200)
    

# HILLSLOPE RELIEF
plt.subplot(224)
freq, bin_edges = np.histogram(BasinHillslopeData.Cht,bins=np.arange(-0.1,0,0.005))
freq_norm = freq.astype(np.float)/float(np.max(freq))
plt.bar(bin_edges[:-1],freq_norm,width=0.005,align='edge',edgecolor='k',linewidth=0.5,color='salmon')
plt.xlabel("Hilltop Curvature (m$^{-1}$)")
plt.ylabel("Normalised Frequency")
plt.xlim(0,-0.1)


# FINALISE THE FIGURE AND SAVE
# add a figure title
plt.suptitle("Basin "+str(Basin)+" Hillslopes")
# check the layout, leave some white space around the edges
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
# save the figure
plt.savefig(ProjectName + "_Basin" + "%02d" % Basin + "_HillslopeHists.png", dpi=300)

### Calculate the dimensionless hillslope morphology within the basin

Recall from the lecture that hillslopes with different lengths will have different relief. Therefore it is useful to calculate the dimensionless properties of hillslopes so that we can compare between different landscapes.

The nonlinear sediment transport equation predicts a relationship between dimensionless erosion rate and dimensionless relief **for steady state hillslopes** of the form:
$$
\begin{equation}
R^* = {{1}\over{E^*}}\left(\sqrt{1+(E^*)^2} - \ln \left[ {{1}\over{2}} \left( 1+\sqrt{1+(E^*)^2}\right) \right] -1 \right) 
\end{equation}
$$

In [ ]:
# Calculate Theoretical Steady-State E* vs R*

# First declare a range of E* values from 10^-1 (0.1) to 10^3 (1000) 
EStarSS = np.logspace(-1,3,1000)

# Calculate RStar as a function of EStar for steady-state hillslopes from equation X
RStarSS = (1./EStarSS)*(np.sqrt(1.+(EStarSS**2.)) - np.log(0.5*(1.+np.sqrt(1+EStarSS**2.))) - 1.)

When $E^*$ < 1, the model predictions are similar to those of a linear sediment flux law, but for $E^*$ > 1 hillslope relief is limited and hillslope gradients approach the critical value $S_C$ and the hillslopes become steep and planar. Conveniently, the relationship between $E^*$ and $R^*$ can also be cast in terms of properties of hillslopes that are measureable from topography. This means that by measuring these properties we can test whether steady-state hillslopes conform with this flux law, or if we are confident the flux law is appropriate we can test whether hillslopes are at steady state:

Now we will calculate $E^*$ and $R^*$ based on the observed topography from our hillslope traces. 

**This next cell of code has been intentionally left blank so that you can complete it yourself.** 

You can access the data for hillslope length with `BasinHillslopeData.Lh`, hilltop curvature with `BasinHillslopeData.Cht` and hillslope gradient with `BasinHillslopeData.S`. The equations for these are:

$$
\begin{equation}
E^* = {{- 2 \:C_{HT} \:L_H}\over{S_C}} %\tag{}
\end{equation}
$$

$$
\begin{equation}
R^* = {{S}\over{S_C}} %\tag{}
\end{equation}
$$

**Complete the cells below to calculate $E^*$ and $R^*$ for your basin**
You may need to look back at the histogram code to figure out how to access the specific data you need within the BasinHillslopeData object.

In [ ]:
#### YOU NEED TO COMPLETE THIS CELL


# Calculate E* vs R* for our hillslope data
EStar = 
RStar = 

Now we will plot the results so we can compare the observed hillslope morphology to the steady-state predictions:

In [ ]:
# setup the figure
# you can modify the figure size here if you wish
fig_width_inches = 6.6
fig_height_inches = 5
fig = plt.figure(figsize=(fig_width_inches,fig_height_inches))
ax = plt.subplot()

# Set up the plot to have a lograthmic scale on both axes
plt.loglog()

# Plot analytical steady-state relationship as a dashed line
ax.plot(EStarSS,RStarSS,'k--',zorder=10)

# Plot the observed hillslope data as a grey scatter plot
ax.plot(EStar,RStar,'.',color=[0.5,0.5,0.5])
    
# Finalise the figure
plt.xlabel('$E^*={{-2\:C_{HT}\:L_H}/{S_C}}$')
plt.ylabel('$R^*=S/S_C$')
plt.xlim(0.1,1000.)
plt.ylim(0.01,1.)
plt.suptitle("Basin "+str(Basin)+" Dimensionless Hillslope Morphology")
plt.tight_layout(rect=[0, 0.03, 1, 0.95])


The results look quite messy. Some of the data sit near the line, and some of the data sit quite far away. **Remember** these are thousands of individual hillslope traces! To try and make a bit more sense of this data we will do some grouping. The hillslope traces record where in the river network the hillslope terminates, and the channel network has been split into 100 m long segments. We will collect all the hillslope data belonging to each channel segment and conduct some statistics. Because we do not know whether the data is normally distributed, you will use the median as our central tendency indicator (instead of mean). To get an idea of the spread of the data, you will use the 16th and 84th percentiles (instead of standard deviation). These concepts should be familiar to you by now but please ask if you're not sure why this is important.

In [ ]:
# get a list of the unique channel segments in the hillslope data
Segments = BasinHillslopeData.StreamID.unique()

# we will create a new dataset to store the results in
# this sets up the new dataset and give names to the columns
SegmentsData = pd.DataFrame(columns=['SegmentNo','EStar','EStarLower','EStarUpper','RStar','RStarLower','RStarUpper','NTraces'])

# loop across all segments in the dataset
for i, Segment in enumerate(Segments):

    # Get segment hillslope data as a new data object
    SegmentHillslopeData = BasinHillslopeData[BasinHillslopeData.StreamID == float(Segment)]

    # Calculate EStar and RStar for the current segment
    TempEs = (-2.*SegmentHillslopeData.Cht*SegmentHillslopeData.Lh)/Sc
    TempRs = SegmentHillslopeData.S/Sc

    # Get the median and quartiles for EStar and RStar
    EStarSeg = TempEs.quantile(0.5)
    EStarSegUpper = TempEs.quantile(0.84)
    EStarSegLower = TempEs.quantile(0.16)
    RStarSeg = TempRs.quantile(0.5)
    RStarSegUpper = TempRs.quantile(0.84)
    RStarSegLower = TempRs.quantile(0.16)

    NTraces = SegmentHillslopeData.size

    #add to the new data frame
    SegmentsData.loc[i] = [Segment,EStarSeg,EStarSegLower,EStarSegUpper,RStarSeg,RStarSegLower,RStarSegUpper,NTraces]

# only keep segments with more than 100 hillslope traces so that we have a good size dataset
SegmentsData = SegmentsData[SegmentsData.NTraces > 100]

Now that you have segmented the hillslope data, let's plot this instead of/on top of the raw hillslope data and see if it provides a clearer picture of what is going on:

In [ ]:
# Plot symbols with error bars with colours but faded (increase alpha)
for i, row in SegmentsData.iterrows(): 
    EStarErr = np.array([[row.EStarLower],[row.EStarUpper]])
    RStarErr = np.array([[row.RStarLower],[row.RStarUpper]])
    ax.plot([row.EStar,row.EStar],RStarErr,'k-', lw=1,alpha=0.5,zorder=9)
    ax.plot(EStarErr,[row.RStar,row.RStar],'k-', lw=1,alpha=0.5,zorder=9)
    ax.plot(row.EStar,row.RStar,'ko',ms=4,zorder=32)

# save the figure
plt.savefig(ProjectName + "_Basin" + "%02d" % Basin + "_EStarRStar.png", dpi=300)

# display the updated plot
fig

## Hillslope morphology along the Bolinas Ridge

You have perhaps observed that hillslope morphology data is quite noisy and quite varied. In order to see through the variability within a basin you will now make a series of plots designed to show how hillslope morphology varies from basin to basin. To do this you will calculate the median, 16th and 84th percentiles of the various hillslope metrics and also the channel steepness data derived by Kirby et al. (2007):

In [ ]:
# we have already loaded the hillslope data, now we need the channel data
Filename = ProjectName+'_MChiSegmented.csv'
ChannelData = pd.read_csv(DataDirectory+Filename)

# Get rid of sections where there is no data
Segments2Remove = ChannelData[ChannelData.chi == -9999].segment_number.unique()
ChannelData = ChannelData[~ChannelData.segment_number.isin(Segments2Remove)]

# get list of basins
Basins = HillslopeData.BasinID.unique()
Basins.sort()

# create a new data object
BasinData = pd.DataFrame(columns=['Basin','CHT','CHT16','CHT84','LH','LH16','LH84','S','S16','S84','MChi','MChi16','MChi84'])
    
# calculate catchment summary data for each basin
for i, Basin in enumerate(Basins):
        
    # isolate hillslope data for this basin
    BasinHillslopeData = HillslopeData[HillslopeData.BasinID == Basin]

    # isolate channel data for this basin
    BasinChannelData = ChannelData[ChannelData.basin_key == i]
    
    # curvature stats
    CHT = BasinHillslopeData.Cht.quantile(0.5)
    CHT16 = BasinHillslopeData.Cht.quantile(0.16)
    CHT84 = BasinHillslopeData.Cht.quantile(0.84)

    # hillslope length stats
    LH = BasinHillslopeData.Lh.quantile(0.5)
    LH16 = BasinHillslopeData.Lh.quantile(0.16)
    LH84 = BasinHillslopeData.Lh.quantile(0.84)

    # slope stats
    S = BasinHillslopeData.S.quantile(0.5)
    S16 = BasinHillslopeData.S.quantile(0.16)
    S84 = BasinHillslopeData.S.quantile(0.84)

    # Channel Steepness stats
    MChi = BasinChannelData.m_chi.quantile(0.5)
    MChi16 = BasinChannelData.m_chi.quantile(0.16)
    MChi84 = BasinChannelData.m_chi.quantile(0.84)

    # add stats to the new data frame
    BasinData.loc[i] = [i,CHT,CHT16,CHT84,LH,LH16,LH84,S,S16,S84,MChi,MChi16,MChi84]


### Compare hillslope morphology along the length of the Bolinas Ridge

Now you will make some plots to show how the hillslope and channel morphology vary along the length of the Bolinas Ridge. In the lecture, we identified that hilltop curvature should be a proxy for erosion rates if the hillslopes are adjusted to their baselevel conditions. Hillslope gradient is also expected to increase with erosion rates, but become limited ***IF*** a process transition to more frequent landsliding has occurred. Think about what sort of relationship you might expect between hilltop curvature and hillslope gradient if this is the case. I suggest sketching out the expected relationship before you make the next plot.

Now go ahead and create a plot of hilltop curvature against hillslope gradient:

In [ ]:
# setup the figure
# you can modify the figure size here if you wish
fig_width_inches = 6.6
fig_height_inches = 5
fig = plt.figure(figsize=(fig_width_inches,fig_height_inches))
ax = plt.subplot()

#set up a colour map for showing basin number
ColourMap = cm.viridis

# Plot symbols with error bars with colours but faded (increase alpha)
for i, row in BasinData.iterrows(): 
    CHTErr = np.array([[row.CHT16],[row.CHT84]])
    SErr = np.array([[row.S16],[row.S84]])
    Colour = float(i)/float(len(BasinData))
    ax.plot([row.CHT,row.CHT],SErr,'-',color=ColourMap(Colour),lw=1,alpha=0.5,zorder=9)
    ax.plot(CHTErr,[row.S,row.S],'-',color=ColourMap(Colour),lw=1,alpha=0.5,zorder=9)
    ax.plot(row.CHT,row.S,'o',color=ColourMap(Colour),ms=5,zorder=32)

# label the axes
plt.xlabel("Hilltop Curvature (m$^{-1}$)")
plt.ylabel("Hillslope Gradient (m/m)")

# add the colour bar
m = cm.ScalarMappable(cmap=ColourMap)
m.set_array(BasinData.Basin)
cbar = plt.colorbar(m)
cbar.set_label("Basin No.")

# save the figure
plt.savefig(ProjectName + "_Cht_S.png", dpi=300)

The results are interesting, and have been colour-coded by basin number from NW to SE to help you to interpret the results. But these results don't allow us to compare the hillslope morphology to the theoretical predictions of the nonlinear sediment flux equation. To do that, we again need to calculate $E^*$ and $R^*$. 

In [ ]:
# Calculate E* vs R* for our hillslope data
EStar = (-2*BasinData.CHT*BasinData.LH)/(Sc)
RStar = BasinData.S/Sc

# Calculate E* vs R* range
EStar16 = (-2*BasinData.CHT16*BasinData.LH16)/(Sc)
RStar16 = BasinData.S16/Sc
EStar84 = (-2*BasinData.CHT84*BasinData.LH84)/(Sc)
RStar84 = BasinData.S84/Sc

# setup the figure
# you can modify the figure size here if you wish
fig_width_inches = 6.6
fig_height_inches = 4.4
fig = plt.figure(figsize=(fig_width_inches,fig_height_inches))
ax = plt.subplot()

#make a logarithmic plot
plt.loglog()

#plot the theoretical relationship between EStar and RStar
ax.plot(EStarSS,RStarSS,'k--',zorder=10)

#set up a colour map for showing basin number
ColourMap = cm.viridis

# Plot symbols with error bars with colours but faded (increase alpha)
for i in range(0,len(Basins)): 
    EStarErr = np.array([[EStar16[i]],[EStar84[i]]])
    RStarErr = np.array([[RStar16[i]],[RStar84[i]]])
    Colour = float(i)/float(len(BasinData))
    ax.plot([EStar[i],EStar[i]],RStarErr,'-',color=ColourMap(Colour),lw=1,alpha=0.5,zorder=9)
    ax.plot(EStarErr,[RStar[i],RStar[i]],'-',color=ColourMap(Colour),lw=1,alpha=0.5,zorder=9)
    ax.plot(EStar[i],RStar[i],'o',color=ColourMap(Colour),ms=5,zorder=32)

# label the axes and limit
plt.xlabel('$E^*={{-2\:C_{HT}\:L_H}/{S_C}}$')
plt.ylabel('$R^*=S/S_C$')
plt.xlim(0.1,100.)
plt.ylim(0.02,1.)

# add the colour bar
m = cm.ScalarMappable(cmap=ColourMap)
m.set_array(BasinData.Basin)
cbar = plt.colorbar(m)
cbar.set_label("Basin No.")

# save the figure
plt.savefig(ProjectName + "_Estar_Rstar.png", dpi=300)

It would seem that hillslope morphology is varying systematically along the length of the Bolinas Ridge. The boundary conditions for the hillslopes are set erosion in the adjacent streams. Kirby et al. (2007) suggested the steepness of the channels increasing along the Bolinas Ridge was reflecting increased erosion in response to a gradient in tectonic uplift rates. Let's use our hillslope data to investigate whether hillslope and channel metrics for erosion rates are comparable by plotting channel steepness ($M_\chi$) against $E^*$:

In [ ]:
# setup the figure
# you can modify the figure size here if you wish
fig_width_inches = 6.6
fig_height_inches = 4.4
fig = plt.figure(figsize=(fig_width_inches,fig_height_inches))
ax = plt.subplot()

#set up a colour map for showing basin number
ColourMap = cm.viridis

# Get MChi data
MChi = BasinData.MChi
MChi16 = BasinData.MChi16
MChi84 = BasinData.MChi84

# Plot symbols with error bars with colours but faded (increase alpha)
for i in range(0,len(Basins)):
    EStarErr = np.array([[EStar16[i]],[EStar84[i]]])
    MChiErr = np.array([[MChi16[i]],[MChi84[i]]])
    Colour = float(i)/float(len(BasinData))
    ax.plot([MChi[i],MChi[i]],EStarErr,'-',color=ColourMap(Colour),lw=1,alpha=0.5,zorder=9)
    ax.plot(MChiErr,[EStar[i],EStar[i]],'-',color=ColourMap(Colour),lw=1,alpha=0.5,zorder=9)
    ax.plot(MChi[i],EStar[i],'o',color=ColourMap(Colour),ms=5,zorder=32)

# label the axes and limit
plt.xlabel('Channel Steepness Index $M_{Chi}$')
plt.ylabel('$E^*={{-2\:C_{HT}\:L_H}/{S_C}}$')

# add the colour bar
m = cm.ScalarMappable(cmap=ColourMap)
m.set_array(BasinData.Basin)
cbar = plt.colorbar(m)
cbar.set_label("Basin No.")

# save the figure
plt.savefig(ProjectName + "_MChi_EStar.png", dpi=300)

<div class="alert alert-block alert-info">
<font color="black">
<h3>ASSESSMENT TASK</h3>
<p>
Describe the distribution of hillslope morphology within an individual catchment and along the length of the Bolinas Ridge. Evaluate how hillslope morphology compares to the steady-state predictions of the non-linear sediment transport equation and to the boundary conditions set by the channel steepness. You may wish to refer to existing academic literature on this site, and on other locations where hillslope morphology has been analysed in this way to help you. You can choose which figures to include, and  remember that your figures will need to have a figure caption. You should approach this task as a short report/essay.
</p>
<p></p>
</font>
</div>